# Data preprocessing
### Información obtenida de terridata PNG
https://terridata.dnp.gov.co/index-app.html#/descargas

### 1. Carga de librerias

In [2]:
pip install clipboard

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import clipboard
import os


### 2. Definicion de funciones

In [324]:
##Función completa datos faltantes de serie de timepo
def completar_datos_serie_tiempo(df,inicio,fin):

    # Convert 'Year' to datetime type with date set to December 31st
    df['Year'] = pd.to_datetime(df['Year'].astype(str) + '-12-31')

    # Set 'Year' and 'Category' as the index
    df.set_index(['Year', 'Category'], inplace=True)

    # Create a new DataFrame with all years and categories from the first year to the last year in the available data
    start_year = inicio
    end_year = fin
    all_years = pd.DataFrame(index=pd.MultiIndex.from_product([[str(y) + '-12-31' for y in range(start_year, end_year+1)], df.index.get_level_values('Category').unique()], names=['Year', 'Category']))


    all_years['join']=all_years.index.get_level_values('Year').astype(str) + '-' + all_years.index.get_level_values('Category').astype(str)
    df['join']=df.index.get_level_values('Year').astype(str) + '-' + df.index.get_level_values('Category').astype(str)


    # Join the existing DataFrame with all_years
    all_years.reset_index(inplace=True)
    all_years['Year'] = pd.to_datetime(all_years['Year'])
    df_complete = pd.merge(all_years, df,on='join',how='left')
    #df_complete_s=df_complete
    df_complete.set_index(['Year', 'Category'], inplace=True)
    df_complete = df_complete.drop('join', axis=1)


    # Obtener la lista de columnas que necesitan ser completadas
    columns_to_interpolate = df_complete.columns.tolist()


    for column in columns_to_interpolate:
        df_complete[column] = df_complete.groupby('Category')[column].apply(lambda x: x.interpolate(limit_direction='both'))
    
    df_complete.reset_index(inplace=True)
    df_complete=df_complete.fillna(0)
    df_complete['Year'] = df_complete['Year'].dt.year
    
    
    return df_complete

In [328]:
def Leer_Archivo(Archivo):

    ruta_archivo = 'C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/'+ Archivo +'.csv'
    
    # Cargar el archivo de texto en un DataFrame
    df = pd.read_csv(ruta_archivo,sep=';',decimal=',',thousands='.')
    
    #df['Dato Numérico'] = df['Dato Numérico'].str.replace('.', '')
    #df['Dato Numérico'] = df['Dato Numérico'].str.replace(',', '.').astype(float)
    
    columnas_deseadas = ['Código Entidad', 'Año','Cat_Tabla','Dato Numérico']
    df = df[columnas_deseadas]
    df = pd.pivot_table(df, values='Dato Numérico', index=['Código Entidad', 'Año'], columns='Cat_Tabla', aggfunc='mean').reset_index()

    df = df.rename(columns={'Código Entidad': 'Category'})
    df = df.rename(columns={'Año': 'Year'})

    
    return df

In [562]:
def unir_archivos_csv_carpeta(carpeta):
    archivos_csv = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.csv')]
    dfs = []
    
    for archivo in archivos_csv:
        ruta_archivo = os.path.join(carpeta, archivo)
        df = pd.read_csv(ruta_archivo)
        dfs.append(df)
    
    dataframe_final = pd.concat(dfs, ignore_index=True)
    return dataframe_final

In [565]:
def merge_files_by_columns(folder_path, join_columns):
    all_data = None

    # Obtener la lista de archivos en la carpeta
    file_list = os.listdir(folder_path)

    for file_name in file_list:
        # Comprobar que los archivos sean de tipo CSV 
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)

            # Leer el archivo CSV en un DataFrame
            df = pd.read_csv(file_path)

            # Realizar la operación de "left join" basado en las columnas especificadas
            if all_data is None:
                # El primer archivo se guarda en la variable all_data
                all_data = df
            else:
                # Realizar el "left join" con los archivos siguientes basado en las columnas de unión
                all_data = all_data.merge(df, on=join_columns, how="left")

    return all_data

### 3. Estandarización de nombre de variables

In [552]:
# 5 Procesamiento archivo excel departamentos
dataframe = pd.read_excel('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/excel/vivienda_servicios_publicos.xlsx',decimal=',',thousands='.')
dataframe = dataframe.drop(0)
promedio_por_grupo = dataframe.groupby('Indicador')['Dato Numérico'].mean()
promedio_por_grupo=promedio_por_grupo.to_frame()
promedio_por_grupo.reset_index(inplace=True)

# Seleccionar la columna que deseas copiar
columna = promedio_por_grupo['Indicador']

# Convertir la columna en una cadena separada por saltos de línea
columna_str = '\n'.join(columna.astype(str))

# Copiar la cadena al portapapeles
clipboard.copy(columna_str)

columna  = pd.DataFrame(columna)

In [458]:
# 1 Procesamiento archivo excel departamentos
Nombre_variable = {'Nombre': ["GestComp", "ResComp", "MDD"]}
Nombre_variable  = pd.DataFrame(Nombre_variable )
Nombre_variable['Nom_completo']='departamento_'+Nombre_variable['Nombre']
merged = columna.merge(Nombre_variable, left_index=True, right_index=True)

dataframe['Cat_Tabla'] = dataframe['Indicador'].map(merged.set_index('Indicador')['Nom_completo'])
dataframe['Año'] = dataframe['Año'].astype(int)
dataframe.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/departamento.csv',sep=';',decimal=',', index=False)

In [462]:
# 2 Procesamiento archivo excel ddes general
nombres_variables = [
    'ruralidad',
    'ley617_2000',
    'codigo_dane',
    'densidad_poblacional',
    'entorno_desarrollo',
    'extension',
    'poblacion_total',
    'region',
    'subregion_sgr',
    'tipologia_dnp',
    'variacion_intercensal_2005_2018'
]

Nombre_variable = {'Nombre': nombres_variables}
Nombre_variable  = pd.DataFrame(Nombre_variable )
Nombre_variable['Nom_completo']='Desp_General_'+ Nombre_variable['Nombre']
merged = columna.merge(Nombre_variable, left_index=True, right_index=True)

dataframe['Cat_Tabla'] = dataframe['Indicador'].map(merged.set_index('Indicador')['Nom_completo'])
dataframe['Año'] = dataframe['Año'].astype(int)
dataframe.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Descripcion_General.csv',sep=';',decimal=',', index=False)

In [469]:
# 3 Procesamiento archivo excel educacion
nombres_variables = [
    'anios_educacion_15mas',
    'cobertura_bruta_total',
    'cobertura_bruta_basica',
    'cobertura_bruta_media',
    'cobertura_bruta_primaria',
    'cobertura_bruta_secundaria',
    'cobertura_bruta_transicion',
    'cobertura_educacion_superior',
    'cobertura_neta_total',
    'cobertura_neta_basica',
    'cobertura_neta_basica_hombres',
    'cobertura_neta_basica_mujeres',
    'cobertura_neta_media',
    'cobertura_neta_primaria',
    'cobertura_neta_secundaria',
    'cobertura_neta_total_hombres',
    'cobertura_neta_total_mujeres',
    'cobertura_neta_transicion',
    'porcentaje_asistencia_5_24',
    'porcentaje_asistencia_rural_5_24',
    'porcentaje_asistencia_urbana_5_24',
    'puntaje_promedio_saber_lectura',
    'puntaje_promedio_saber_matematicas',
    'tasa_analfabetismo',
    'tasa_analfabetismo_rural',
    'tasa_analfabetismo_urbana',
    'tasa_analfabetismo_15mas',
    'tasa_desercion_oficial_basica_media',
    'tasa_repitencia_oficial_basica_media',
    'tasa_transito_educacion_superior'
]

Nombre_variable = {'Nombre': nombres_variables}
Nombre_variable  = pd.DataFrame(Nombre_variable )
Nombre_variable['Nom_completo']='Educacion_'+ Nombre_variable['Nombre']
merged = columna.merge(Nombre_variable, left_index=True, right_index=True)

dataframe['Cat_Tabla'] = dataframe['Indicador'].map(merged.set_index('Indicador')['Nom_completo'])
dataframe['Año'] = dataframe['Año'].astype(int)
dataframe.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Educacion.csv',sep=';',decimal=',', index=False)

In [474]:
# 4 Procesamiento archivo excel finanzas

nombres_variables = [
    '%ingresos_funcionamiento',
    '%ingresos_recursos_propios',
    '%ingresos_transferencias',
    '%gasto_total_inversion',
    'agua',
    'agua_potable',
    'alimentacion_escolar',
    'componente_eficacia',
    'componente_eficiencia',
    'componente_sin_medidas_SMSCE',
    'componente_transparencia',
    'credito_interno_externo',
    'cultura',
    'deporte',
    'deficit_ahorro_corriente',
    'deficit_superavit_total',
    'educacion',
    'financiamiento',
    'fonpet',
    'funcionamiento',
    'gastos_corrientes',
    'gastos_corrientes_per_capita',
    'gastos_capital_inversion',
    'gastos_funcionamiento_per_capita',
    'gastos_totales',
    'gastos_totales_per_capita',
    'ingresos_corrientes',
    'ingresos_corrientes_per_capita',
    'ingresos_capital',
    'ingresos_no_tributarios',
    'ingresos_no_tributarios_per_capita',
    'ingresos_per_capita_impuesto_industria_comercio',
    'ingresos_per_capita_impuesto_predial',
    'ingresos_totales',
    'ingresos_totales_per_capita',
    'ingresos_tributarios',
    'ingresos_tributarios_per_capita',
    'intereses_deuda_publica',
    'libre_destinacion',
    'libre_inversion',
    'num_proyectos_contratados',
    'num_proyectos_sin_contratar',
    'num_proyectos_terminados',
    'num_total_proyectos',
    '%inversion_recursos_propios',
    'primera_infancia',
    'proposito_general',
    'recursos_balance_variacion_depositos',
    'regalias_per_capita',
    'resguardos_indigenas',
    'respaldo_deuda',
    'ribereños',
    'saldo_directas',
    'saldo_especificas',
    'saldo_total',
    'salud',
    'total_SGP',
    'total_asignaciones_SGR_2012',
    'total_asignaciones_SGR_2013_2014',
    'total_asignaciones_SGR_2015_2016',
    'total_asignaciones_SGR_2017_2018',
    'total_recursos_asignados_per_capita_SGP',
    'total_recursos_asignados_per_capita_SGR',
    'transferencias_ingresos_corrientes',
    'transferencias_nacionales_per_capita_SGP',
    'transferencias_per_capita_ingresos_corrientes',
    'valor_proyectos_contratados',
    'valor_proyectos_sin_contratar',
    'valor_proyectos_terminados',
    'valor_numero_total_proyectos',
    'indice_gestion_proyectos_regalias'
]


Nombre_variable = {'Nombre': nombres_variables}
Nombre_variable  = pd.DataFrame(Nombre_variable )
Nombre_variable['Nom_completo']='Finanzas_'+ Nombre_variable['Nombre']
merged = columna.merge(Nombre_variable, left_index=True, right_index=True)

dataframe['Cat_Tabla'] = dataframe['Indicador'].map(merged.set_index('Indicador')['Nom_completo'])
dataframe['Año'] = dataframe['Año'].astype(int)
dataframe.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Finanzas.csv',sep=';',decimal=',', index=False)


In [479]:
# 5 Procesamiento archivo excel de justicia

nombres_variables = [
    'acuerdo_asuntos_especiales',
    'acuerdo_asuntos_familiares',
    'acuerdo_derecho_civil_comercial',
    'acuerdo_derecho_comunitario',
    'acuerdo_derecho_contencioso_administrativo',
    'acuerdo_derecho_familia',
    'acuerdo_derecho_propiedad_intelectual',
    'acuerdo_derecho_laboral',
    'acuerdo_derecho_penal',
    'acuerdo_todas_ramas_derecho',
    'acuerdo_area_derecho_no_definida',
    'asistentes_actividades_pedagogicas_jornadas_moviles_justicia',
    'asuntos_no_conciliables_asuntos_especiales',
    'asuntos_no_conciliables_asuntos_familiares',
    'asuntos_no_conciliables_derecho_civil_comercial',
    'asuntos_no_conciliables_derecho_comunitario',
    'asuntos_no_conciliables_derecho_contencioso_administrativo',
    'asuntos_no_conciliables_derecho_familia',
    'asuntos_no_conciliables_derecho_propiedad_intelectual',
    'asuntos_no_conciliables_derecho_laboral',
    'asuntos_no_conciliables_derecho_penal',
    'asuntos_no_conciliables_todas_ramas_derecho',
    'asuntos_no_conciliables_area_derecho_no_definida',
    'barrera_acceso_justicia_1_SLJ',
    'barrera_acceso_justicia_2_SLJ',
    'barrera_acceso_justicia_3_SLJ',
    'barrera_acceso_justicia_4_SLJ',
    'barrera_acceso_justicia_5_SLJ',
    'casos_atendidos_jornadas_moviles_justicia',
    'casos_atendidos_justicia_jornadas_moviles_justicia',
    'casos_atendidos_otros_temas_jornadas_moviles_justicia',
    'conflictividad_1_SLJ',
    'conflictividad_2_SLJ',
    'conflictividad_3_SLJ',
    'conflictividad_4_SLJ',
    'conflictividad_5_SLJ',
    'inasistencia_asuntos_especiales',
    'inasistencia_asuntos_familiares',
    'inasistencia_derecho_civil_comercial',
    'inasistencia_derecho_comunitario',
    'inasistencia_derecho_contencioso_administrativo',
    'inasistencia_derecho_familia',
    'inasistencia_derecho_propiedad_intelectual',
    'inasistencia_derecho_laboral',
    'inasistencia_derecho_penal',
    'inasistencia_todas_ramas_derecho',
    'inasistencia_area_derecho_no_definida',
    'jornadas_moviles_justicia_realizadas',
    'otros_asuntos_especiales',
    'otros_asuntos_familiares',
    'otros_derecho_civil_comercial',
    'otros_derecho_comunitario',
    'otros_derecho_contencioso_administrativo',
    'otros_derecho_familia',
    'otros_derecho_propiedad_intelectual',
    'otros_derecho_laboral',
    'otros_derecho_penal',
    'otros_todas_ramas_derecho',
    'otros_area_derecho_no_definida',
    'resultados_conciliacion_resueltas_1_semana_menos',
    'resultados_conciliacion_resueltas_mas_1_año',
    'resultados_conciliacion_resueltas_entre_1_semana_1_mes',
    'resultados_conciliacion_resueltas_entre_2_3_meses',
    'resultados_conciliacion_resueltas_entre_4_meses_1_año',
    'resultados_conciliacion_sin_finalizar'
]

Nombre_variable = {'Nombre': nombres_variables}
Nombre_variable  = pd.DataFrame(Nombre_variable )
Nombre_variable['Nom_completo']='Justicia_'+ Nombre_variable['Nombre']
merged = columna.merge(Nombre_variable, left_index=True, right_index=True)

dataframe['Cat_Tabla'] = dataframe['Indicador'].map(merged.set_index('Indicador')['Nom_completo'])
dataframe['Año'] = dataframe['Año'].astype(int)
dataframe.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Justicia.csv',sep=';',decimal=',', index=False)

In [484]:
# 6 Procesamiento archivo excel de Censo

nombres_variables = [
    'pob_hombres_05_09',
    'pob_hombres_10_14',
    'pob_hombres_15_19',
    'pob_hombres_20_24',
    'pob_hombres_25_29',
    'pob_hombres_30_34',
    'pob_hombres_35_39',
    'pob_hombres_40_44',
    'pob_hombres_45_49',
    'pob_hombres_50_54',
    'pob_hombres_55_59',
    'pob_hombres_60_64',
    'pob_hombres_65_69',
    'pob_hombres_70_74',
    'pob_hombres_75_79',
    'pob_hombres_00_04',
    'pob_hombres_80_mas',
    'pob_mujeres_05_09',
    'pob_mujeres_10_14',
    'pob_mujeres_15_19',
    'pob_mujeres_20_24',
    'pob_mujeres_25_29',
    'pob_mujeres_30_34',
    'pob_mujeres_35_39',
    'pob_mujeres_40_44',
    'pob_mujeres_45_49',
    'pob_mujeres_50_54',
    'pob_mujeres_55_59',
    'pob_mujeres_60_64',
    'pob_mujeres_65_69',
    'pob_mujeres_70_74',
    'pob_mujeres_75_79',
    'pob_mujeres_00_04',
    'pob_mujeres_80_mas',
    'pob_indigena',
    'pob_negra_mulata_afrocolombiana',
    'pob_palenquero',
    'pob_raizal',
    'pob_rom',
    'pob_rural',
    'pob_total_hombres',
    'pob_total_mujeres',
    'pob_urbana',
    'pob_etnica_total',
    'porc_pob_hombres_05_09',
    'porc_pob_hombres_10_14',
    'porc_pob_hombres_15_19',
    'porc_pob_hombres_20_24',
    'porc_pob_hombres_25_29',
    'porc_pob_hombres_30_34',
    'porc_pob_hombres_35_39',
    'porc_pob_hombres_40_44',
    'porc_pob_hombres_45_49',
    'porc_pob_hombres_50_54',
    'porc_pob_hombres_55_59',
    'porc_pob_hombres_60_64',
    'porc_pob_hombres_65_69',
    'porc_pob_hombres_70_74',
    'porc_pob_hombres_75_79',
    'porc_pob_hombres_00_04',
    'porc_pob_hombres_80_mas',
    'porc_pob_mujeres_05_09',
    'porc_pob_mujeres_10_14',
    'porc_pob_mujeres_15_19',
    'porc_pob_mujeres_20_24',
    'porc_pob_mujeres_25_29',
    'porc_pob_mujeres_30_34',
    'porc_pob_mujeres_35_39',
    'porc_pob_mujeres_40_44',
    'porc_pob_mujeres_45_49',
    'porc_pob_mujeres_50_54',
    'porc_pob_mujeres_55_59',
    'porc_pob_mujeres_60_64',
    'porc_pob_mujeres_65_69',
    'porc_pob_mujeres_70_74',
    'porc_pob_mujeres_75_79',
    'porc_pob_mujeres_00_04',
    'porc_pob_mujeres_80_mas',
    'porc_pob_indigena',
    'porc_pob_negra_mulata_afrocolombiana',
    'porc_pob_palenquero',
    'porc_pob_raizal',
    'porc_pob_rom',
    'porc_pob_etnica_total',
    'porc_pob_hombres',
    'porc_pob_mujeres',
    'porc_pob_rural',
    'porc_pob_urbana',
    'proyeccion_pob_0_4',
    'proyeccion_pob_10_14',
    'proyeccion_pob_5_9'
]

Nombre_variable = {'Nombre': nombres_variables}
Nombre_variable  = pd.DataFrame(Nombre_variable )
Nombre_variable['Nom_completo']='Censo_'+ Nombre_variable['Nombre']
merged = columna.merge(Nombre_variable, left_index=True, right_index=True)

dataframe['Cat_Tabla'] = dataframe['Indicador'].map(merged.set_index('Indicador')['Nom_completo'])
dataframe['Año'] = dataframe['Año'].astype(int)
dataframe.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Censo.csv',sep=';',decimal=',', index=False)

In [489]:
# 7 Procesamiento archivo excel de mercado laboral

nombres_variables = [
    'empresas_generadoras_empleo_10k_habitantes',
    'participacion_trabajadores_cotizantes_hombres',
    'participacion_trabajadores_cotizantes_mujeres',
    'participacion_trabajadores_cotizantes_sin_reporte_sexo',
    'participacion_trabajadores_cotizantes_17_anos_menos',
    'participacion_trabajadores_cotizantes_18_28_anos',
    'participacion_trabajadores_cotizantes_29_40_anos',
    'participacion_trabajadores_cotizantes_41_50_anos',
    'participacion_trabajadores_cotizantes_51_59_anos',
    'participacion_trabajadores_cotizantes_60_anos_mas',
    'participacion_trabajadores_cotizantes_sin_reporte_edad',
    'porcentaje_personas_ocupadas_formalmente_poblacion_total',
    'trabajadores_cotizantes_hombres',
    'trabajadores_cotizantes_mujeres',
    'trabajadores_cotizantes_sin_reporte_sexo',
    'trabajadores_cotizantes_promedio_mensual',
    'trabajadores_cotizantes_17_anos_menos',
    'trabajadores_cotizantes_18_28_anos',
    'trabajadores_cotizantes_29_40_anos',
    'trabajadores_cotizantes_41_50_anos',
    'trabajadores_cotizantes_51_59_anos',
    'trabajadores_cotizantes_60_anos_mas',
    'trabajadores_cotizantes_sin_reporte_edad'
]

Nombre_variable = {'Nombre': nombres_variables}
Nombre_variable  = pd.DataFrame(Nombre_variable )
Nombre_variable['Nom_completo']='Mercado_Laboral_'+ Nombre_variable['Nombre']
merged = columna.merge(Nombre_variable, left_index=True, right_index=True)

dataframe['Cat_Tabla'] = dataframe['Indicador'].map(merged.set_index('Indicador')['Nom_completo'])
dataframe['Año'] = dataframe['Año'].astype(int)
dataframe.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Mercado_Laboral.csv',sep=';',decimal=',', index=False)

In [500]:
# 9 Procesamiento archivo excel Poblacion
nombres_variables = [
    'GrupoA',
    'GrupoB',
    'GrupoC',
    'GrupoD',
    'Hombres70PEP',
    'H0-7HombresPEP',
    'H18-29HombresPEP',
    'H30-39HombresPEP',
    'H40-49HombresPEP',
    'H50-59HombresPEP',
    'H60-69HombresPEP',
    'H8-17HombresPEP',
    'Mujeres70PEP',
    'M0-7MujeresPEP',
    'M18-29MujeresPEP',
    'M30-39MujeresPEP',
    'M40-49MujeresPEP',
    'M50-59MujeresPEP',
    'M60-69MujeresPEP',
    'M8-17MujeresPEP',
    'NumResguardosIndigenas',
    'PobH00-04',
    'PobH05-09',
    'PobH10-14',
    'PobH15-19',
    'PobH20-24',
    'PobH25-29',
    'PobH30-34',
    'PobH35-39',
    'PobH40-44',
    'PobH45-49',
    'PobH50-54',
    'PobH55-59',
    'PobH60-64',
    'PobH65-69',
    'PobH70-74',
    'PobH75-79',
    'PobH80+',
    'PobM00-04',
    'PobM05-09',
    'PobM10-14',
    'PobM15-19',
    'PobM20-24',
    'PobM25-29',
    'PobM30-34',
    'PobM35-39',
    'PobM40-44',
    'PobM45-49',
    'PobM50-54',
    'PobM55-59',
    'PobM60-64',
    'PobM65-69',
    'PobM70-74',
    'PobM75-79',
    'PobM80+',
    'PobEtnicoResguardos',
    'PobIndigena',
    'PobMigranteVenezuelaPEP',
    'PobNegraAfrocolombiana',
    'PobPalenquero',
    'PobRaizal',
    'PobRom',
    'PobRural',
    'PobTotalHombres',
    'PobTotalMujeres',
    'PobUrbana',
    'PobEtnicaTotal',
    'PorcH70+PEP',
    'PorcH0-7PEP',
    'PorcH18-29PEP',
    'PorcH30-39PEP',
    'PorcH40-49PEP',
    'PorcH50-59PEP',
    'PorcH60-69PEP',
    'PorcH8-17PEP',
    'PorcM70+PEP',
    'PorcM0-7PEP',
    'PorcM18-29PEP',
    'PorcM30-39PEP',
    'PorcM40-49PEP',
    'PorcM50-59PEP',
    'PorcM60-69PEP',
    'PorcM8-17PEP',
    'PorcMigranteVenezuela',
    'PorcPob80+',
    'PorcPobH00-04',
    'PorcPobH05-09',
    'PorcPobH10-14',
    'PorcPobH15-19',
    'PorcPobH20-24',
    'PorcPobH25-29',
    'PorcPobH30-34',
    'PorcPobH35-39',
    'PorcPobH40-44',
    'PorcPobH45-49',
    'PorcPobH50-54',
    'PorcPobH55-59',
    'PorcPobH60-64',
    'PorcPobH65-69',
    'PorcHombres70-74',
    'PorcHombres75-79',
    'PorcHombres80mas',
    'PorcJovenes14-28',
    'PorcMujeres00-04',
    'PorcMujeres05-09',
    'PorcMujeres10-14',
    'PorcMujeres15-19',
    'PorcMujeres20-24',
    'PorcMujeres25-29',
    'PorcMujeres30-34',
    'PorcMujeres35-39',
    'PorcMujeres40-44',
    'PorcMujeres45-49',
    'PorcMujeres50-54',
    'PorcMujeres55-59',
    'PorcMujeres60-64',
    'PorcMujeres65-69',
    'PorcMujeres70-74',
    'PorcMujeres75-79',
    'PorcMujeres80mas',
    'PorcPobEtnicoResguardos',
    'PorcPob0-4',
    'PorcPob10-14',
    'PorcPob15-19',
    'PorcPob20-24',
    'PorcPob25-29',
    'PorcPob30-34',
    'PorcPob35-39',
    'PorcPob40-44',
    'PorcPob45-49',
    'PorcPob5-9',
    'PorcPob50-54',
    'PorcPob55-59',
    'PorcPob60-64',
    'PorcPob65-69',
    'PorcPob70-74',
    'PorcPob75-79',
    'PctMenores15',
    'PctPobMigranteInterna',
    'PctPobRegularPEP',
    'PctPobHombres',
    'PctPobMujeres',
    'PctPobRural',
    'PctPobUrbana',
    'ProyVenezolanos',
    'ProyPob80mas',
    'ProyPob0-4',
    'ProyPob10-14',
    'ProyPob15-19',
    'ProyPob20-24',
    'ProyPob25-29',
    'ProyPob30-34',
    'ProyPob35-39',
    'ProyPob40-44',
    'ProyPob45-49',
    'ProyPob5-9',
    'ProyPob50-54',
    'ProyPob55-59',
    'ProyPob60-64',
    'ProyPob65-69',
    'ProyPob70-74',
    'ProyPob75-79',
    'Punt_SISBEN_MaxCabecera',
    'Punt_SISBEN_MaxCentroPoblado',
    'Punt_SISBEN_MaxRuralDisperso',
    'Punt_SISBEN_MaxTotal',
    'Punt_SISBEN_MinCabecera',
    'Punt_SISBEN_MinCentroPoblado',
    'Punt_SISBEN_MinRuralDisperso',
    'Punt_SISBEN_MinTotal',
    'Punt_SISBEN_PromCabecera',
    'Punt_SISBEN_PromCentroPoblado',
    'Punt_SISBEN_PromRuralDisperso',
    'PuntajePromTotal',
    'TotalFichas',
    'TotalHogares',
    'TotalPersonas',
    'TotalHombresPEP',
    'TotalMujeresPEP',
    'TotalRegistrosFichas',
    'TotalRegistrosHogares',
    'TotalRegistrosPersonas',
    'TotalRegistrosValidosFichas',
    'TotalRegistrosValidosHogares',
    'TotalRegistrosValidosPersonas'

]

Nombre_variable = {'Nombre': nombres_variables}
Nombre_variable  = pd.DataFrame(Nombre_variable )
Nombre_variable['Nom_completo']='Poblacion_'+ Nombre_variable['Nombre']
merged = columna.merge(Nombre_variable, left_index=True, right_index=True)

dataframe['Cat_Tabla'] = dataframe['Indicador'].map(merged.set_index('Indicador')['Nom_completo'])
dataframe['Año'] = dataframe['Año'].astype(int)
dataframe.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Poblacion.csv',sep=';',decimal=',', index=False)

In [543]:
# 10 Procesamiento archivo excel Pobreza

nombres_variables = [
    'CoefGini',
    'IPMCabecera',
    'IPMRural',
    'IncidPobMonetaria',
    'IncidPobMonetariaActual',
    'IncidPobMonetariaExtrema',
    'IncidPobMonetariaExtremaActual',
    'IncidPobMonetariaJefFemenina',
    'IncidPobMonetariaJefMasculina',
    'IncidPobMonetariaHombres',
    'IncidPobMonetariaHombresActual',
    'IncidPobMonetariaMujeres',
    'IncidPobMonetariaMujeresActual',
    'IncidPobMonetariaExtremaJefFemenina',
    'IncidPobMonetariaExtremaJefMasculina',
    'IncidPobMonetariaExtremaHombres',
    'IncidPobMonetariaExtremaHombresActual',
    'IncidPobMonetariaExtremaMujeres',
    'IncidPobMonetariaExtremaMujeresActual',
    'PoblacionMiseria',
    'PoblacionMiseriaRural',
    'PoblacionMiseriaUrbana',
    'INBI-Rural',
    'INBI-Urbana',
    'IPM'
]

Nombre_variable = {'Nombre': nombres_variables}
Nombre_variable  = pd.DataFrame(Nombre_variable )
Nombre_variable['Nom_completo']='Probreza_'+ Nombre_variable['Nombre']
merged = columna.merge(Nombre_variable, left_index=True, right_index=True)

dataframe['Cat_Tabla'] = dataframe['Indicador'].map(merged.set_index('Indicador')['Nom_completo'])
dataframe['Año'] = dataframe['Año'].astype(int)
dataframe.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pobreza.csv',sep=';',decimal=',', index=False)

In [539]:
# 12 Procesamiento archivo excel salud

nombres_variables = [
    'AfilRegEspeciales',
    'AfilSGSSS',
    'AfilRegContributivo',
    'AfilRegSubsidiado',
    'CobVacTripleViral',
    'CobVacDPT',
    'CobRegSubsidiado',
    'CobVacPentavalente',
    'IncidEnfChagas',
    'IncidTuberculosis',
    'IncidVIH',
    'IncidVIHHombre',
    'IncidVIHMujer',
    'IncidDengue',
    'MortMenores5EDA',
    'MortMenores5IRA',
    'MortDesnutricionMenores5',
    'MortAsocVIH_SIDA',
    'MujSumAnticoncep',
    'PersDiscapacidad',
    'PobPobreNoAtendida',
    'PorcNacVivosCtrlPrenatal',
    'PorcNacVivosBajoPeso',
    'PorcNacVivosMadres1417',
    'PorcNacVivosMadres1826',
    'PorcNacVivosMadresMen14',
    'PromControlesPrenatales',
    'RazMortMaterna42Dias',
    'TasaMortAccidentesTranspTerrestre',
    'TasaMortAccTranspTerrestreHombres',
    'TasaMortAccTranspTerrestreMujeres',
    'TasaMortLesAutoinfligSuicidios',
    'TasaMortLesAutoinfligSuicidiosHombre',
    'TasaMortLesAutoinfligSuicidiosMujer',
    'TasaMortTumorMalignoMama',
    'TasaMortTumorMalignoProstata',
    'TasaMortTumorMalignoCuelloUtero',
    'TasaFecundidad',
    'TasaFecundidad1519',
    'TasaFecundidad1014',
    'TasaFecundidad1019',
    'TasaMortalidad',
    'TasaMortalidadInfantil1Anio',
    'TasaMortalidadInfantil5Anios',
    'TasaMortalidadNeonatal',
    'TasaMortalidadPrematuraDiabetes',
    'TasaMortalidadPrematuraEnfCirculatorio',
    'TasaNegligenciaAbandono'
]


Nombre_variable = {'Nombre': nombres_variables}
Nombre_variable  = pd.DataFrame(Nombre_variable )
Nombre_variable['Nom_completo']='salud_'+ Nombre_variable['Nombre']
merged = columna.merge(Nombre_variable, left_index=True, right_index=True)

dataframe['Cat_Tabla'] = dataframe['Indicador'].map(merged.set_index('Indicador')['Nom_completo'])
dataframe['Año'] = dataframe['Año'].astype(int)
dataframe.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/salud.csv',sep=';',decimal=',', index=False)

In [546]:
# 13 Procesamiento archivo excel seg_fluvial

nombres_variables = [
    'ArmasIncautadas',
    'CasosPirateria',
    'CasosTerrorismo',
    'ContrabandoBienesFungibles',
    'ContrabandoBienesMuebles',
    'ContrabandoBienesNoFungibles',
    'EmbarcacionesRescatadas_ArtefactoNaval',
    'EmbarcacionesRescatadas_BotesLanchas',
    'EmbarcacionesRescatadas_Motonaves',
    'EmbarcacionesRescatadas_YatesVeleros',
    'EspeciesExtincion_Aves',
    'EspeciesExtincion_FaunaPeces',
    'EspeciesExtincion_Flora',
    'EspeciesExtincion_Herpetofauna',
    'EspeciesExtincion_Mamiferos',
    'EspeciesIncautadas_Camaron',
    'EspeciesIncautadas_Caracol',
    'EspeciesIncautadas_Langosta',
    'EspeciesMaderables_Ceiba',
    'EspeciesMaderables_Madera',
    'EspeciesMaderables_Otros',
    'EspeciesMaderables_Sajo',
    'EspeciesMaderables_Sande',
    'EspeciesMaderables_TangareMangle',
    'HurtosMar_ArtefactoNaval',
    'HurtosMar_BotesLanchas',
    'HurtosMar_Motonaves',
    'HurtosMar_YatesVeleros',
    'Incautaciones_ArmasAcompanamiento',
    'Incautaciones_ArmasCortoAlcance',
    'Incautaciones_ArmasLargoAlcance',
    'Incautaciones_Bazuco',
    'Incautaciones_ClorhidratoCocaina',
    'Incautaciones_Heroina',
    'Incautaciones_Marihuana',
    'Incautaciones_MunicionEstrategica',
    'Incautaciones_MunicionMayor',
    'Incautaciones_MunicionMenor',
    'Incautaciones_PrecursorCocaina',
    'Incautaciones_SustanciasExplosivas',
    'MercanciasContrabandoIncautadas',
    'OperacionesBusquedaRescate',
    'PersonasEvacuadas_BusquedaRescate',
    'PersonasRescatadas_BusquedaRescate',
    'RoboManoArmada'
]

Nombre_variable = {'Nombre': nombres_variables}
Nombre_variable  = pd.DataFrame(Nombre_variable )
Nombre_variable['Nom_completo']='seg_fluvial_'+ Nombre_variable['Nombre']
merged = columna.merge(Nombre_variable, left_index=True, right_index=True)

dataframe['Cat_Tabla'] = dataframe['Indicador'].map(merged.set_index('Indicador')['Nom_completo'])
dataframe['Año'] = dataframe['Año'].astype(int)
dataframe.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/seg_fluvial.csv',sep=';',decimal=',', index=False)

In [550]:
# 14 Procesamiento archivo excel seguridad

nombres_variables = [
    'CuartaMedidaCorrectiva',
    'PrimeraMedidaCorrectiva',
    'ProporcionOtrasMedidasCorrectivas',
    'QuintaMedidaCorrectiva',
    'SegundaMedidaCorrectiva',
    'TasaExtorsion',
    'TasaHomicidiosAccidentesTransito',
    'TasaHomicidiosHombres',
    'TasaHomicidiosMujeres',
    'TasaHomicidiosAreasRurales',
    'TasaHomicidiosAreasUrbanas',
    'TasaHomicidios',
    'TasaHurtoPersonas',
    'TasaHurtoComun',
    'TasaLesionesAccidentesTransito',
    'TasaSecuestro',
    'TasaViolenciaInterpersonal',
    'TasaViolenciaIntrafamiliar',
    'TerceraMedidaCorrectiva'
]


Nombre_variable = {'Nombre': nombres_variables}
Nombre_variable  = pd.DataFrame(Nombre_variable )
Nombre_variable['Nom_completo']='seguridad_'+ Nombre_variable['Nombre']
merged = columna.merge(Nombre_variable, left_index=True, right_index=True)

dataframe['Cat_Tabla'] = dataframe['Indicador'].map(merged.set_index('Indicador')['Nom_completo'])
dataframe['Año'] = dataframe['Año'].astype(int)
dataframe.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/seguridad.csv',sep=';',decimal=',', index=False)

In [553]:
# 15 Procesamiento archivo excel vivienda_servicios_publicos

nombres_variables = [
    'CoberturaGasNaturalUrbanoCenso',
    'CoberturaAseoRuralCenso',
    'CoberturaAseoUrbanaCenso',
    'CoberturaEnergiaElectricaCenso',
    'CoberturaEnergiaElectricaRuralCenso',
    'CoberturaEnergiaElectricaUrbanaCenso',
    'CoberturaGasNaturalCenso',
    'CoberturaGasNaturalRuralCenso',
    'CoberturaInternetCenso',
    'CoberturaInternetRuralCenso',
    'CoberturaInternetUrbanaCenso',
    'CoberturaAcueductoCenso',
    'CoberturaAcueductoREC',
    'CoberturaAcueductoRuralCenso',
    'CoberturaAcueductoRuralREC',
    'CoberturaAcueductoUrbanaCenso',
    'CoberturaAcueductoUrbanaREC',
    'CoberturaAlcantarilladoCenso',
    'CoberturaAlcantarilladoREC',
    'CoberturaAlcantarilladoRuralCenso',
    'CoberturaAlcantarilladoRuralREC',
    'CoberturaAlcantarilladoUrbanaCenso',
    'CoberturaAlcantarilladoUrbanaREC',
    'CoberturaAseoCenso',
    'CoberturaAseoREC',
    'CoberturaAseoRuralREC',
    'CoberturaAseoUrbanaREC',
    'CoberturaEnergiaElectricaRural',
    'DeficitCualitativoViviendaCenso',
    'DeficitCuantitativoViviendaCenso',
    'IndigenaAcueducto',
    'NegroAcueducto',
    'PalenqueroAcueducto',
    'PenetracionBandaAncha',
    'RaizalSanAndresProvidenciaAcueducto',
    'RomAcueducto'
]


Nombre_variable = {'Nombre': nombres_variables}
Nombre_variable  = pd.DataFrame(Nombre_variable )
Nombre_variable['Nom_completo']='vivienda_servicios_publicos_'+ Nombre_variable['Nombre']
merged = columna.merge(Nombre_variable, left_index=True, right_index=True)

dataframe['Cat_Tabla'] = dataframe['Indicador'].map(merged.set_index('Indicador')['Nom_completo'])
dataframe['Año'] = dataframe['Año'].astype(int)
dataframe.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/vivienda_servicios_publicos.csv',sep=';',decimal=',', index=False)

### 4. Limpieza de datos y organización de frame por categoria

In [333]:
#1_Leer y transformar achivo de Ambinete
Tipo_dato='Ambiente'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)

In [334]:
#2_Leer y transformar achivo de Ordenamiento_Territorial
Tipo_dato='Ordenamiento_Territorial'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)

In [343]:
#3_Leer y transformar achivo de Desempeño
Tipo_dato='desempeño'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)

In [352]:
#4_Leer y transformar achivo de Economia
Tipo_dato='Economia'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)

In [459]:
#5_Leer y transformar achivo de departamento
Tipo_dato='departamento'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)

In [464]:
#6_Leer y transformar achivo de Descripcion_General
Tipo_dato='Descripcion_General'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)

In [470]:
#7_Educacion
Tipo_dato='Educacion'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)

In [476]:
#8_Finanzas
Tipo_dato='Finanzas'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)



In [481]:
#9_Justicia
Tipo_dato='Justicia'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)


In [486]:
#10_Censo
Tipo_dato='Censo'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)


In [491]:

#11_Mercado_Laboral
Tipo_dato='Mercado_Laboral'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)


In [502]:
#12_Poblacion
Tipo_dato='Poblacion'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)


In [556]:
#13_Pobreza
Tipo_dato='Pobreza'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)


In [557]:
#14_salud 
Tipo_dato='salud'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)


In [558]:
#15_seg_fluvial 
Tipo_dato='seg_fluvial'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)


In [559]:
#16_seguridad 
Tipo_dato='seguridad'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)


In [560]:
#17_vivienda_servicios_publicos
Tipo_dato='vivienda_servicios_publicos'
df=Leer_Archivo(Tipo_dato)
df=completar_datos_serie_tiempo(df,2008,2022)
df.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed/'+Tipo_dato+'.csv', index=False)


### 5. Generacion del archivo Integrado

In [567]:

folder_path = "C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Pre_Processed"
join_columns = ["Category","Year"]

merged_data = merge_files_by_columns(folder_path, join_columns)



In [569]:
merged_data .to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Base_0.csv', index=False)


In [6]:
ruta_archivo = 'C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Terridata/Base_0.csv'
    
# Cargar el archivo de texto en un DataFrame
df = pd.read_csv(ruta_archivo,sep=',')

In [11]:
df[df['seg_fluvial_ArmasIncautadas'].isna()].head()

,Year,Category,Ambiente_absorcion,Ambiente_acumulado,Ambiente_area_amenazada,Ambiente_area_bosque,Ambiente_area_deforestada,Ambiente_area_ecosistemas,Ambiente_area_humedales,Ambiente_area_manglares,...,vivienda_servicios_publicos_CoberturaInternetRuralCenso,vivienda_servicios_publicos_CoberturaInternetUrbanaCenso,vivienda_servicios_publicos_DeficitCualitativoViviendaCenso,vivienda_servicios_publicos_DeficitCuantitativoViviendaCenso,vivienda_servicios_publicos_IndigenaAcueducto,vivienda_servicios_publicos_NegroAcueducto,vivienda_servicios_publicos_PalenqueroAcueducto,vivienda_servicios_publicos_PenetracionBandaAncha,vivienda_servicios_publicos_RaizalSanAndresProvidenciaAcueducto,vivienda_servicios_publicos_RomAcueducto
2,2008,5001,0.0,2799.38,31939.03,9512.38,1080.82,131.62,131.62,0.00,...,0.0,0.0,12.98,2.02,0.0,0.0,0.0,11.3725,0.0,0.0
128,2008,8001,0.0,2023.97,174.66,72.81,44.94,3116.82,3024.85,91.97,...,0.0,0.0,20.90,5.13,0.0,0.0,0.0,8.7450,0.0,0.0
151,2008,11001,0.0,1284.37,69402.33,8083.00,142.00,96043.73,5112.69,0.00,...,0.0,0.0,10.21,3.86,0.0,0.0,0.0,11.6125,0.0,0.0
153,2008,13001,0.0,1962.40,19052.47,2812.47,501.75,23101.43,17568.39,3457.70,...,0.0,0.0,28.43,10.30,0.0,0.0,0.0,6.4850,0.0,0.0
200,2008,15001,0.0,902.57,3633.85,85.77,20.47,807.03,0.00,0.00,...,0.0,0.0,12.49,1.46,0.0,0.0,0.0,5.7225,0.0,0.0


In [ ]:
valores_faltantes = valores_faltantes[valores_faltantes != 0]